# CHAPTER 11 Time Series

时间序列指能在任何能在时间上观测到的数据。很多时间序列是有固定频率（fixed frequency）的，意思是数据点会遵照某种规律定期出现，比如每15秒，每5分钟，或每个月。时间序列也可能是不规律的（irregular），没有一个固定的时间规律。如何参照时间序列数据取决于我们要做什么样的应用，我们可能会遇到下面这些：

* Timestamps（时间戳），具体的某一个时刻

* Fixed periods（固定的时期），比如2007年的一月，或者2010年整整一年

* Intervals of time（时间间隔），通常有一个开始和结束的时间戳。Periods（时期）可能被看做是Intervals（间隔）的一种特殊形式。

* Experiment or elapsed time（实验或经过的时间）；每一个时间戳都是看做是一个特定的开始时间（例如，在放入烤箱后，曲奇饼的直径在每一秒的变化程度）

这一章主要涉及前三个类型。

> pandas也支持基于timedeltas的index，本书不会对timedelta index做介绍，感兴趣的可以查看pandas的文档。

## 11.1 Date and Time Data Types and Tools

python有标准包用来表示时间和日期数据。datetime, time, calendar，这些模块经常被使用。datetime.datetime类型，或简单写为datetime，被广泛使用：

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
now = datetime.now()
now

datetime.datetime(2018, 1, 27, 10, 47, 4, 395198)

In [3]:
now.year, now.month, now.day

(2018, 1, 27)

datetime能保存日期和时间到微妙级别。timedelta表示两个不同的datetime对象之间的时间上的不同：

In [4]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(926, 56700)

In [6]:
delta.days, delta.seconds

(926, 56700)

我们可以在一个datetime对象上，添加或减少一个或多个timedelta，这样可以产生新的变化后的对象："

In [7]:
from datetime import timedelta

In [8]:
start = datetime(2011, 1, 7)
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [9]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

下表汇总了一些datetime模块中的数据类型：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/wqo6m.png)

### Converting Between String and Datetime

我们可以对datetime对象，以及pandas的Timestamp对象进行格式化，这部分之后会介绍，使用str或strftime方法，传入一个特定的时间格式就能进行转换：

In [10]:
stamp = datetime(2011, 1, 3)
str(stamp)

'2011-01-03 00:00:00'

In [11]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

In [13]:
stamp.strftime('%F')

'2011-01-03'

下表是关于日期时间类型的格式：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/r98dw.png)

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/bc9e8.png)

我们可以利用上面的format codes（格式码；时间日期格式）把字符串转换为日期，这要用到datetime.strptime:

In [15]:
value = '2011-01-03'

datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [16]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

对于一个时间格式，使用datetime.strptime来解析日期是很好的方法。但是，如果每次都要写格式的话很烦人，尤其是对于一些比较常见的格式。在这种情况下，我们可以使用第三方库dateutil中的parser.parse方法（这个库会在安装pandas的时候自动安装）：

In [17]:
from dateutil.parser import parse

In [18]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

dateutil能够解析很多常见的时间表示格式：

In [19]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

在国际上，日在月之前是很常见的（译者：美国是把月放在日前面的），所以我们可以设置dayfirst=True来指明最前面的是否是日：

In [20]:
parse('6/12/2001', dayfirst=True)

datetime.datetime(2001, 12, 6, 0, 0)

pandas通常可以用于处理由日期组成的数组，不论是否是DataFrame中的行索引或列。to_datetime方法能解析很多不同种类的日期表示。标准的日期格式，比如ISO 8601，能被快速解析：

In [21]:
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

In [22]:
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

还能处理一些应该被判断为缺失的值（比如None, 空字符串之类的）：

In [24]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [25]:
idx[2]

NaT

In [27]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

Nat(Not a Time)在pandas中，用于表示时间戳为空值（null value）。

> dateutil.parse是一个很有用但不完美的工具。它可能会把一些字符串识别为日期，例如，'42'就会被解析为2042年加上今天的日期。

datetime对象还有一些关于地区格式（locale-specific formatting）的选项，用于处理不同国家或不同语言的问题。例如，月份的缩写在德国和法国，与英语是不同的。下表列出一些相关的选项：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/gp2fy.png)

## 11.2 Time Series Basics

在pandas中，一个基本的时间序列对象，是一个用时间戳作为索引的Series，在pandas外部的话，通常是用python 字符串或datetime对象来表示的：

In [28]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8), 
         datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02   -1.151882
2011-01-05    1.331437
2011-01-07   -0.313103
2011-01-08    0.349794
2011-01-10    0.344231
2011-01-12    0.055980
dtype: float64

上面的转化原理是，datetime对象被放进了DatetimeIndex:

In [29]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

像其他的Series一行，数值会自动按时间序列索引进行对齐：

In [30]:
ts[::2]

2011-01-02   -1.151882
2011-01-07   -0.313103
2011-01-10    0.344231
dtype: float64

In [31]:
ts + ts[::2]

2011-01-02   -2.303764
2011-01-05         NaN
2011-01-07   -0.626207
2011-01-08         NaN
2011-01-10    0.688462
2011-01-12         NaN
dtype: float64

ts[::2]会在ts中，每隔两个元素选一个元素。

pandas中的时间戳，是按numpy中的datetime64数据类型进行保存的，可以精确到纳秒的级别：

In [32]:
ts.index.dtype

dtype('<M8[ns]')

DatetimeIndex的标量是pandas的Timestamp对象：

In [33]:
ts.index[0]

Timestamp('2011-01-02 00:00:00')

### Indexing, Selection, Subsetting

当我们基于标签进行索引和选择时，时间序列就像是pandas.Series：

In [34]:
ts

2011-01-02   -1.151882
2011-01-05    1.331437
2011-01-07   -0.313103
2011-01-08    0.349794
2011-01-10    0.344231
2011-01-12    0.055980
dtype: float64

In [36]:
stamp = ts.index[2]
ts[stamp]

-0.31310325944913037

为了方便，我们可以直接传入一个字符串用来表示日期：

In [37]:
ts['1/10/2011']

0.34423120430645771

In [38]:
ts['20110110']

0.34423120430645771

对于比较长的时间序列，我们可以直接传入一年或一年一个月，来进行数据选取：

In [42]:
longer_ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
longer_ts.head()

2000-01-01    0.721759
2000-01-02   -0.702178
2000-01-03    1.638362
2000-01-04   -0.374051
2000-01-05   -1.769831
Freq: D, dtype: float64

In [43]:
longer_ts['2001'].head()

2001-01-01    1.723546
2001-01-02    0.306078
2001-01-03    0.349107
2001-01-04   -1.340365
2001-01-05   -0.005353
Freq: D, dtype: float64

这里，字符串'2001'就直接被解析为一年，然后选中这个时期的数据。我们也可以指定月份：

In [44]:
longer_ts['2001-05'].head()

2001-05-01   -0.240385
2001-05-02    0.008372
2001-05-03    0.513756
2001-05-04    0.478063
2001-05-05    0.718189
Freq: D, dtype: float64

利用datetime进行切片（slicing）也没问题：

In [45]:
ts[datetime(2011, 1, 7)]

-0.31310325944913037

因为大部分时间序列是按年代时间顺序来排列的，我们可以用时间戳来进行切片，选中一段范围内的时间：

In [46]:
ts['1/6/2011':'1/11/2011']

2011-01-07   -0.313103
2011-01-08    0.349794
2011-01-10    0.344231
dtype: float64

记住，这种方式的切片得到的只是原来数据的一个视图，如果我们在切片的结果上进行更改的的，原来的数据也会变化。

有一个相等的实例方法（instance method）也能切片，truncate，能在两个日期上，对Series进行切片：

In [47]:
ts.truncate(after='1/9/2011')

2011-01-02   -1.151882
2011-01-05    1.331437
2011-01-07   -0.313103
2011-01-08    0.349794
dtype: float64

所有这些都适用于DataFrame，我们对行进行索引：

In [48]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')

long_df = pd.DataFrame(np.random.randn(100, 4),
                       index=dates,
                       columns=['Colorado', 'Texas',
                                'New York', 'Ohio'])

long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,0.902208,2.102457,0.631011,0.495778
2001-05-09,0.353968,-1.942254,-0.640874,0.975342
2001-05-16,0.420884,0.898728,-0.680524,-1.107961
2001-05-23,0.296874,-1.003480,1.027778,-0.145218
2001-05-30,-1.152153,-1.216092,0.525163,-1.890308


### Time Series with Duplicate Indices

在某些数据中，可能会遇到多个数据在同一时间戳下的情况：

In [49]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000', 
                          '1/2/2000', '1/3/2000'])

dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [50]:
dup_ts.index.is_unique

False

对这个时间序列取索引的的话， 要么得到标量，要么得到切片，这取决于时间戳是否是重复的：

In [51]:
dup_ts['1/3/2000']

4

In [52]:
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

假设我们想要聚合那些有重复时间戳的数据，一种方法是用groupby，设定level=0：

In [53]:
dup_ts.groupby(level=0).mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [54]:
dup_ts.groupby(level=0).count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3 Date Ranges, Frequencies, and Shifting

普通的时间序列通常是不规律的，但我们希望能有一个固定的频度，比如每天，每月，或没15分钟，即使有一些缺失值也没关系。幸运的是，pandas中有一套方法和工具来进行重采样，推断频度，并生成固定频度的日期范围。例如，我们可以把样本时间序列变为固定按日的频度，需要调用resample：

In [55]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8), 
         datetime(2011, 1, 10), datetime(2011, 1, 12)]

ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02   -0.242984
2011-01-05    0.493147
2011-01-07   -1.017858
2011-01-08    0.571822
2011-01-10   -1.052385
2011-01-12    0.228062
dtype: float64

In [56]:
resampler = ts.resample('D')
resampler

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

这里的'D'表示按日的频度（daily frequency）。

关于频度（frequency）和重采样（resampling）的转换，会在11.6进行具体介绍，这里我们展示一些基本的用法。

### Generating Date Ranges

之前虽然用过，但没有做解释，其实pandas.date_range是用来生成DatetimeIndex的，使用时要根据频度来指明长度：

In [57]:
pd.date_range('2017-04-01', '2017-06-01')

DatetimeIndex(['2017-04-01', '2017-04-02', '2017-04-03', '2017-04-04',
               '2017-04-05', '2017-04-06', '2017-04-07', '2017-04-08',
               '2017-04-09', '2017-04-10', '2017-04-11', '2017-04-12',
               '2017-04-13', '2017-04-14', '2017-04-15', '2017-04-16',
               '2017-04-17', '2017-04-18', '2017-04-19', '2017-04-20',
               '2017-04-21', '2017-04-22', '2017-04-23', '2017-04-24',
               '2017-04-25', '2017-04-26', '2017-04-27', '2017-04-28',
               '2017-04-29', '2017-04-30', '2017-05-01', '2017-05-02',
               '2017-05-03', '2017-05-04', '2017-05-05', '2017-05-06',
               '2017-05-07', '2017-05-08', '2017-05-09', '2017-05-10',
               '2017-05-11', '2017-05-12', '2017-05-13', '2017-05-14',
               '2017-05-15', '2017-05-16', '2017-05-17', '2017-05-18',
               '2017-05-19', '2017-05-20', '2017-05-21', '2017-05-22',
               '2017-05-23', '2017-05-24', '2017-05-25', '2017-05-26',
      

默认，date_range会生成按日频度的时间戳。如果我们只传入一个开始或一个结束时间，还必须传入一个数字来表示时期：

In [58]:
pd.date_range(start='2012-04-01', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [59]:
pd.date_range(end='2012-06-01', periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

开始和结束的日期，严格指定了用于生成日期索引（date index）的边界。例如，如果我们希望日期索引包含每个月的最后一个工作日，我们要设定频度为'BM'(business end of month，每个月的最后一个工作日，更多频度可以看下面的表格)，而且只有在这个日期范围内的日期会被包含进去：

In [60]:
pd.date_range('2000-01-01', '2000-12-01', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

时间序列频度：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/v4ae4.png)

date_range会默认保留开始或结束的时间戳：

In [62]:
pd.date_range('2012-05-02 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

有些时候我们的时间序列数据带有小时，分，秒这样的信息，但我们想要让这些时间戳全部归一化到午夜（normalized to midnight, 即晚上0点），这个时候要用到normalize选项：

In [64]:
nor_date = pd.date_range('2012-05-02 12:56:31', periods=5, normalize=True)
nor_date 

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

In [65]:
nor_date[0]

Timestamp('2012-05-02 00:00:00', freq='D')

可以看到小时，分，秒全部变为0

### Frequencies and Date Offsets

pandas中的频度由一个基本频度（base frequency）和一个乘法器（multiplier）组成。基本频度通常用一个字符串别名（string alias）来代表，比如'M'表示月，'H'表示小时。对每一个基本频度，还有一个被称之为日期偏移（date offset）的对象。例如，小时频度能用Hour类来表示：

In [2]:
from pandas.tseries.offsets import Hour, Minute

In [3]:
hour = Hour()
hour

<Hour>

通过传入一个整数，我们可以定义一个乘以偏移的乘法（a multiple of an offset）：

In [4]:
four_hours = Hour(4)
four_hours

<4 * Hours>

在很多情况下，我们不需要创建这些对象，而是使用字符串别名，比如'H'或'4H'。在频度前加一个整数，就能作为一个乘法器：

In [5]:
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4H')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

很多偏移(offset)还能和加法结合：

In [6]:
Hour(2) + Minute(30)

<150 * Minutes>

同样的，我们可以传入频度字符串，比如'1h30min'，这种表达也能被解析：

In [7]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

#### Week of month dates

一个有用的类（class）是月中的第几周（Week of month），用WOM表示。比如我们想得到每个月的第三个星期五：

In [9]:
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')
rng

DatetimeIndex(['2012-01-20', '2012-02-17', '2012-03-16', '2012-04-20',
               '2012-05-18', '2012-06-15', '2012-07-20', '2012-08-17'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

In [10]:
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

### Shifting (Leading and Lagging) Data （偏移（提前与推后）数据）

偏移（shifting）表示按照时间把数据向前或向后推移。Series和DataFrame都有一个shift方法实现偏移，索引（index）不会被更改：

In [11]:
ts = pd.Series(np.random.randn(4), index=pd.date_range('1/1/2000', periods=4, freq='M'))
ts

2000-01-31   -1.114814
2000-02-29   -0.081377
2000-03-31   -0.015766
2000-04-30    1.100279
Freq: M, dtype: float64

In [12]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -1.114814
2000-04-30   -0.081377
Freq: M, dtype: float64

In [13]:
ts.shift(-2)

2000-01-31   -0.015766
2000-02-29    1.100279
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

当我们进行位移的时候，就像上面这样会引入缺失值。

shift的一个普通的用法是计算时间序列的百分比变化，可以表示为：

In [14]:
ts / ts.shift(1) - 1

2000-01-31          NaN
2000-02-29    -0.927004
2000-03-31    -0.806258
2000-04-30   -70.787307
Freq: M, dtype: float64

因为普通的shift不会对index进行修改，一些数据会被丢弃。因此如果频度是已知的，可以把频度传递给shift，这样的话时间戳会自动变化：

In [15]:
ts

2000-01-31   -1.114814
2000-02-29   -0.081377
2000-03-31   -0.015766
2000-04-30    1.100279
Freq: M, dtype: float64

In [16]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -1.114814
2000-04-30   -0.081377
Freq: M, dtype: float64

In [18]:
ts.shift(2, freq='M')

2000-03-31   -1.114814
2000-04-30   -0.081377
2000-05-31   -0.015766
2000-06-30    1.100279
Freq: M, dtype: float64

其他一些频度也可以导入，能让我们前后移动数据：

In [19]:
ts.shift(3, freq='D')

2000-02-03   -1.114814
2000-03-03   -0.081377
2000-04-03   -0.015766
2000-05-03    1.100279
dtype: float64

In [21]:
ts.shift(1, freq='90T')
# T表示分钟。

2000-01-31 01:30:00   -1.114814
2000-02-29 01:30:00   -0.081377
2000-03-31 01:30:00   -0.015766
2000-04-30 01:30:00    1.100279
Freq: M, dtype: float64

#### Shifting dates with offsets

pandas的日期偏移（date offset）能被用于datetime或Timestamp对象：

In [22]:
from pandas.tseries.offsets import  Day, MonthEnd

In [23]:
now = datetime(2011, 11, 7)
now + 3 * Day()

Timestamp('2011-11-10 00:00:00')

如果我们添加一个像MonthEnd这样的anchored offset(依附偏移；锚点位置)，日期会根据频度规则进行递增：

In [24]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [25]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

依附偏移可以让日期向前或向后滚动，利用rollforward和rollback方法：

In [26]:
offset = MonthEnd()

In [27]:
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [28]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

一个比较创造性的日期偏移（date offset）用法是配合groupby一起用：

In [29]:
ts = pd.Series(np.random.randn(20), index=pd.date_range('1/15/2000', periods=20, freq='4d'))
ts

2000-01-15    1.292201
2000-01-19    0.621502
2000-01-23   -1.240256
2000-01-27    0.042905
2000-01-31   -2.314308
2000-02-04    0.175818
2000-02-08   -1.104119
2000-02-12    1.205473
2000-02-16   -0.338267
2000-02-20    0.573082
2000-02-24   -1.091848
2000-02-28    0.774478
2000-03-03   -0.317629
2000-03-07    1.205586
2000-03-11    0.716205
2000-03-15    0.228994
2000-03-19    0.485559
2000-03-23    2.056588
2000-03-27   -1.787605
2000-03-31    0.658911
Freq: 4D, dtype: float64

In [31]:
(1.292201 + 0.621502 - 1.240256 + 0.042905 - 2.314308) / 5

-0.3195912

In [32]:
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.319591
2000-02-29    0.027802
2000-03-31    0.405826
dtype: float64

一个简单且快捷的方式是用resample(11.6会进行更详细的介绍)：

In [33]:
ts.resample('M').mean()

2000-01-31   -0.319591
2000-02-29    0.027802
2000-03-31    0.405826
Freq: M, dtype: float64

## 11.4 Time Zone Handling

> 格林威治标准时间GMT

> 十七世纪，格林威治皇家天文台为了海上霸权的扩张计画而进行天体观测。1675年旧皇家观测所(Old Royal Observatory) 正式成立，到了1884年决定以通过格林威治的子午线作为划分地球东西两半球的经度零度。观测所门口墙上有一个标志24小时的时钟，显示当下的时间，对全球而言，这里所设定的时间是世界时间参考点，全球都以格林威治的时间作为标准来设定时间，这就是我们耳熟能详的「格林威治标准时间」(Greenwich Mean Time，简称G.M.T.)的由来，标示在手表上，则代表此表具有两地时间功能，也就是同时可以显示原居地和另一个国度的时间。

> 世界协调时间UTC

> 多数的两地时间表都以GMT来表示，但也有些两地时间表上看不到GMT字样，出现的反而是UTC这3个英文字母，究竟何谓UTC？事实上，UTC指的是Coordinated Universal Time－ 世界协调时间（又称世界标准时间、世界统一时间），是经过平均太阳时(以格林威治时间GMT为准)、地轴运动修正后的新时标以及以「秒」为单位的国际原子时所综合精算而成的时间，计算过程相当严谨精密，因此若以「世界标准时间」的角度来说，UTC比GMT来得更加精准。其误差值必须保持在0.9秒以内，若大于0.9秒则由位于巴黎的国际地球自转事务中央局发布闰秒，使UTC与地球自转周期一致。所以基本上UTC的本质强调的是比GMT更为精确的世界时间标准，不过对于现行表款来说，GMT与UTC的功能与精确度是没有差别的。

时区可以理解为UTC的偏移（offset），例如，在夏令时，纽约时间落后于UTC时间四个小时，而在一年的其他时间里，纽约时间落后于UTC时间五个小时。

在python中，时区信息来自第三方的pytz库，这个库利用的是奥尔森数据库，这个数据库汇集了世界时区信息。这个信息对于历史数据很重要，因为夏令时（daylight saving time，DST）的交接日（transition date）取决于当地政府的心血来潮。在美国，自1900年后，夏令时的交接日已经被改了很多次。

关于pytz库的更多信息，需要查看相关的文档。本书中pandas包含了一些pytz的功能，除了时区的名字，其他的API都不用去查。时区名字可以通过下面的方法获得：

In [34]:
import pytz

In [35]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

想要从pytz中得到一个时区对象（time zone object），使用pytz.timezone:

In [36]:
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

### Time Zone Localization and Conversion

默认的，pandas中的时间序列是time zone naive（朴素时区）。例如，考虑下面的时间序列：

In [37]:
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-09 09:30:00   -1.080150
2012-03-10 09:30:00   -1.603125
2012-03-11 09:30:00   -0.664898
2012-03-12 09:30:00   -0.754545
2012-03-13 09:30:00    0.452152
2012-03-14 09:30:00   -1.321236
Freq: D, dtype: float64

索引的tz部分是None：

In [40]:
print(ts.index.tz)

None


In [41]:
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

使用tz_localize方法，可以实现从朴素到本地化（naive to localized）的转变：

In [42]:
ts

2012-03-09 09:30:00   -1.080150
2012-03-10 09:30:00   -1.603125
2012-03-11 09:30:00   -0.664898
2012-03-12 09:30:00   -0.754545
2012-03-13 09:30:00    0.452152
2012-03-14 09:30:00   -1.321236
Freq: D, dtype: float64

In [43]:
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00   -1.080150
2012-03-10 09:30:00+00:00   -1.603125
2012-03-11 09:30:00+00:00   -0.664898
2012-03-12 09:30:00+00:00   -0.754545
2012-03-13 09:30:00+00:00    0.452152
2012-03-14 09:30:00+00:00   -1.321236
Freq: D, dtype: float64

In [44]:
ts_utc.index.tz

<UTC>

一旦时间序列被定位到某个时区，那么它就可以被转换为任何其他时区，使用tz_convert：

In [48]:
ts_utc.tz_convert('America/New_York')

2012-03-09 04:30:00-05:00   -1.080150
2012-03-10 04:30:00-05:00   -1.603125
2012-03-11 05:30:00-04:00   -0.664898
2012-03-12 05:30:00-04:00   -0.754545
2012-03-13 05:30:00-04:00    0.452152
2012-03-14 05:30:00-04:00   -1.321236
Freq: D, dtype: float64

在处理时间序列的时候，我们可以先把时间定位到纽约时间，然后转换到柏林时间：

In [49]:
ts_eastern = ts.tz_localize('America/New_York')
ts_eastern

2012-03-09 09:30:00-05:00   -1.080150
2012-03-10 09:30:00-05:00   -1.603125
2012-03-11 09:30:00-04:00   -0.664898
2012-03-12 09:30:00-04:00   -0.754545
2012-03-13 09:30:00-04:00    0.452152
2012-03-14 09:30:00-04:00   -1.321236
Freq: D, dtype: float64

In [50]:
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00   -1.080150
2012-03-10 14:30:00+00:00   -1.603125
2012-03-11 13:30:00+00:00   -0.664898
2012-03-12 13:30:00+00:00   -0.754545
2012-03-13 13:30:00+00:00    0.452152
2012-03-14 13:30:00+00:00   -1.321236
Freq: D, dtype: float64

In [51]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00   -1.080150
2012-03-10 15:30:00+01:00   -1.603125
2012-03-11 14:30:00+01:00   -0.664898
2012-03-12 14:30:00+01:00   -0.754545
2012-03-13 14:30:00+01:00    0.452152
2012-03-14 14:30:00+01:00   -1.321236
Freq: D, dtype: float64

tz_localize和tz_convert也是DatetimeIndex上的实例方法（instance methods）：

In [52]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

讲朴素的时间戳进行本地化，还会检查夏令时转换期附近是否有模糊的或不存在的时间。

### Operations with Time Zone−Aware Timestamp Objects

和时间序列或日期范围（date ranges）相似，单独的Timestamp object（时间戳对象）也能从朴素（即无时区）本地化为有时区的日期，然后就可以转换为其他时区了：

In [55]:
stamp = pd.Timestamp('2011-03-12 04:00')
stamp

Timestamp('2011-03-12 04:00:00')

In [56]:
stamp_utc = stamp.tz_localize('utc')

In [57]:
stamp_utc.tz_convert('America/New_York')

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

在创建Timestamp的时候，我们可以传递一个时区：

In [58]:
stamp_moscow = pd.Timestamp('2011-03-12 04:00', tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

有时区的Timestamp对象内部存储了一个UTC时间戳，这个值是从Unix纪元（即1907年1月1日）到现在的纳秒；这个UTC值在即使换了不同的时区，也是不变的：

In [59]:
stamp_utc.value

1299902400000000000

In [60]:
stamp_utc.tz_convert('America/New_York').value

1299902400000000000

在使用pandas的DateOffset对象进行算数运算的时候，如果夏令时存在，pandas也会考虑进去。这里我们构建一个时间戳，正好出现在夏令时转换前。首先，在变为夏令时的前30分钟：

In [61]:
from pandas.tseries.offsets import Hour

In [62]:
stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [63]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [64]:
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [65]:
stamp + 2 * Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

### Operations Between Diferent Time Zones

如果两个不同时区的时间序列被合并，那么结果为UTC。因为时间戳是以UTC为背后机制的，这种变化是直接的，不需要手动转换：

In [66]:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')

ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-07 09:30:00   -0.742168
2012-03-08 09:30:00    0.056187
2012-03-09 09:30:00   -0.828335
2012-03-12 09:30:00   -0.045583
2012-03-13 09:30:00    0.542602
2012-03-14 09:30:00    0.275819
2012-03-15 09:30:00    0.713506
2012-03-16 09:30:00    1.199439
2012-03-19 09:30:00    0.886545
2012-03-20 09:30:00   -0.406259
Freq: B, dtype: float64

In [67]:
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
result = ts1 + ts2
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

## 11.5 Periods and Period Arithmetic

Periods（周期）表示时间跨度（timespans），比如天，月，季，年。Period类表示的就是这种数据类型，构建的时候需要用字符串或整数，以及一个频度（关于频度的代码可以看11.4中的表格）：

In [68]:
p = pd.Period(2007, freq='A-DEC')
p

Period('2007', 'A-DEC')

在这个例子里，Period对象代表了整个2007年一年的时间跨度，从1月1日到12月31日。在Period对象上进行加减，会有和对频度进行位移（shifting）一样的效果：

In [69]:
p + 5

Period('2012', 'A-DEC')

In [70]:
p - 2

Period('2005', 'A-DEC')

如果两个周期有相同的频度，二者的区别就是它们之间有多少个单元（units）：

In [71]:
pd.Period('2014', freq='A-DEC') - p

7

固定范围的周期（Regular ranges of periods）可以通过period_range函数创建：

In [72]:
rng = pd.period_range('2000-01-01', '2000-06-03', freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

PeriodIndex类能存储周期组成的序列，而且可以作为任何pandas数据结构中的轴索引（axis index）：

In [73]:
pd.Series(np.random.randn(6), index=rng)

2000-01    0.002404
2000-02   -0.143109
2000-03    0.941658
2000-04   -0.786431
2000-05   -0.442674
2000-06    0.927204
Freq: M, dtype: float64

如果我们有字符串组成的数组，可以使用PeriodIndex类：

In [74]:
values = ['2001Q3', '2002Q2', '2003Q1']

index = pd.PeriodIndex(values, freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

### Period Frequency Conversion

通过使用asfreq方法，Periods和PeriodIndex对象能被转换为其他频度。例如，假设我们有一个年度期间（annual period），并且想要转换为月度期间（monthly period），做法非常直观：

In [75]:
p = pd.Period('2007', freq='A-DEC')
p

Period('2007', 'A-DEC')

In [76]:
p.asfreq('M', how='start')

Period('2007-01', 'M')

In [77]:
p.asfreq('M', how='end')

Period('2007-12', 'M')

我们可以认为Period('2007', freq='A-DEC')是某种指向时间跨度的光标，而这个时间跨度被细分为月度期间。可以看下面的图示：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/bomb8.png)

如果一个财政年度（fiscal year）是在1月结束，而不是12月，那么对应的月度期间会不一样：

In [78]:
p = pd.Period('2007', 'A-JUN')
p

Period('2007', 'A-JUN')

In [80]:
p.asfreq('M', how='start')

Period('2006-07', 'M')

In [81]:
p.asfreq('M', how='end')

Period('2007-06', 'M')

当我们转换高频度为低频度时，pandas会根据 subperiod（次周期；子周期）的归属来决定superperiod（超周期；母周期）。例如，在A-JUN频度中，月份Aug-2007其实是个2008周期的一部分：

In [82]:
p = pd.Period('Aug-2007', 'M')
p

Period('2007-08', 'M')

In [83]:
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

整个PeriodIndex对象或时间序列可以被转换为一样的语义（semantics）：

In [84]:
rng = pd.period_range('2006', '2009', freq='A-DEC')

ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2006   -1.413937
2007   -0.865398
2008    2.560413
2009    0.761691
Freq: A-DEC, dtype: float64

In [85]:
ts.asfreq('M', how='start')

2006-01   -1.413937
2007-01   -0.865398
2008-01    2.560413
2009-01    0.761691
Freq: M, dtype: float64

这里，年度周期可以用月度周期替换，对应的第一个月也会包含在每个年度周期里。如果我们想要每年的最后一个工作日的话，可以使用'B'频度，并指明我们想要周期的结尾：

In [86]:
ts.asfreq('B', how='end')

2006-12-29   -1.413937
2007-12-31   -0.865398
2008-12-31    2.560413
2009-12-31    0.761691
Freq: B, dtype: float64

#### Converting Timestamps to Periods (and Back)

用时间戳作为索引的Series和DataFrame对象，可以用to_period方法转变为周期：

In [87]:
rng = pd.date_range('2000-01-01', periods=3, freq='M')

ts = pd.Series(np.random.randn(3), index=rng)
ts

2000-01-31   -0.355556
2000-02-29    1.422708
2000-03-31    2.047486
Freq: M, dtype: float64

In [88]:
pts = ts.to_period()
pts

2000-01   -0.355556
2000-02    1.422708
2000-03    2.047486
Freq: M, dtype: float64

因为周期是不重复的时间跨度（non-overlapping timespans），一个时间戳只能属于一个有指定频度的单独周期。尽管默认情况下新的PeriodIndex的频度会从时间戳中来推测，但我们也可以自己设定想要的频度。结果中有重复的周期也没有关系：

In [89]:
rng = pd.date_range('1/29/2000', periods=6, freq='D')
rng

ts2 = pd.Series(np.random.randn(6), index=rng)
ts2

2000-01-29   -0.161911
2000-01-30   -0.355091
2000-01-31    0.591161
2000-02-01   -0.112989
2000-02-02    1.751235
2000-02-03   -1.095837
Freq: D, dtype: float64

In [91]:
ts2.to_period('M')

2000-01-29   -0.161911
2000-01-30   -0.355091
2000-01-31    0.591161
2000-02-01   -0.112989
2000-02-02    1.751235
2000-02-03   -1.095837
Freq: D, dtype: float64

想转换回时间戳的话，使用to_timestamp:

In [92]:
pts = ts2.to_period()
pts

pts.to_timestamp(how='end')

2000-01-29   -0.161911
2000-01-30   -0.355091
2000-01-31    0.591161
2000-02-01   -0.112989
2000-02-02    1.751235
2000-02-03   -1.095837
Freq: D, dtype: float64

#### Creating a PeriodIndex from Arrays

有固定频度的数据集，有时会在很多列上存储时间跨度信息。例如，在下面的宏观经济数据及上，年度和季度在不同的列：

In [93]:
data = pd.read_csv("./Data/Example/macrodata.csv")
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [94]:
data.year[:5]

0    1959.0
1    1959.0
2    1959.0
3    1959.0
4    1960.0
Name: year, dtype: float64

In [95]:
data.quarter[:5]

0    1.0
1    2.0
2    3.0
3    4.0
4    1.0
Name: quarter, dtype: float64

通过把这些数组传递给PeriodIndex，并指定频度，我们可以把这些合并得到一个新的DataFrame：

In [96]:
index = pd.PeriodIndex(year=data.year, quarter=data.quarter, freq='Q-DEC')
index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')

In [97]:
data.index = index

In [98]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
1959Q1,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [99]:
data.infl[:5]

1959Q1    0.00
1959Q2    2.34
1959Q3    2.74
1959Q4    0.27
1960Q1    2.31
Freq: Q-DEC, Name: infl, dtype: float64

### 11.6 Resampling and Frequency Conversion

重采样（Resampling）指的是把时间序列的频度变为另一个频度的过程。把高频度的数据变为低频度叫做降采样（downsampling），把低频度变为高频度叫做增采样（upsampling）。并不是所有的重采样都会落入上面这几个类型，例如，把W-WED（weekly on Wednesday）变为W-FRI，既不属于降采样，也不属于增采样。

pandas对象自带resampe方法，用于所有的频度变化。resample有一个和groupby类似的API；我们可以用resample来对数据进行分组，然后调用聚合函数（aggregation function）:

In [100]:
rng = pd.date_range('2000-01-01', periods=100, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.head()

2000-01-01   -0.163646
2000-01-02    0.108344
2000-01-03    0.157173
2000-01-04   -0.962373
2000-01-05   -0.869862
Freq: D, dtype: float64

In [101]:
ts.resample('M').mean()

2000-01-31   -0.143827
2000-02-29    0.020387
2000-03-31   -0.075315
2000-04-30   -0.331616
Freq: M, dtype: float64

In [102]:
ts.resample('M', kind='period').mean()

2000-01   -0.143827
2000-02    0.020387
2000-03   -0.075315
2000-04   -0.331616
Freq: M, dtype: float64

resample是一个灵活且高效的方法，可以用于处理大量的时间序列。下面是一些相关的选项：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/nwc0s.png)

#### Downsampling

把数据聚合为规律、低频度是一个很普通的时间序列任务。用于处理的数据不必是有固定频度的；我们想要设定的频度会定义箱界（bin edges），根据bin edges会把时间序列分割为多个片段，然后进行聚合。例如，转换为月度，比如'M'或'BM'，我们需要把数据以月为间隔进行切割。每一个间隔都是半开放的（half-open）；一个数据点只能属于一个间隔，所有间隔的合集，构成整个时间范围（time frame）。当使用resample去降采样数据的时候，有很多事情需要考虑：

* 在每个间隔里，哪一边要闭合
* 怎样对每一个聚合的bin贴标签，可以使用间隔的开始或结束

为了演示一下，下面用一个一分钟的数据来举例：

In [106]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')

ts = pd.Series(np.arange(12), index=rng)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int64

假设我们想要按5分钟一个数据块来进行聚合，然后对每一个组计算总和：

In [105]:
ts.resample('5min').sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int64

我们传入的频度定义了每个bin的边界按5分钟递增。默认的是bin的左边界是闭合的，所以00:00值是属于00:00到00:05间隔的。设定closed='right'，会让间隔的右边闭合：

In [108]:
ts.resample('5min', closed='right').sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int64

默认，每一个bin的左边的时间戳，会被用来作为结果里时间序列的标签。通过设置label='right'，我们可以使用bin右边的时间戳来作为标签：

默认，每一个bin的左边的时间戳，会被用来作为结果里时间序列的标签。通过设置label='right'，我们可以使用bin右边的时间戳来作为标签：

In [109]:
ts.resample('5min', closed='right', label='right').sum()

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5T, dtype: int64

可以看下图方便理解：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/d770h.png)

最后，我们可能想要对结果的索引进行位移，比如在右边界减少一秒。想要实现的话，传递一个字符串或日期偏移给loffset：

In [110]:
ts.resample('5min', closed='right', label='right', loffset='-1s').sum()

1999-12-31 23:59:59     0
2000-01-01 00:04:59    15
2000-01-01 00:09:59    40
2000-01-01 00:14:59    11
Freq: 5T, dtype: int64

我们也可以使用shift方法来实现上面loffset的效果。

#### Open-High-Low-Close (OHLC) resampling

> Open-High-Low-Close: 开盘-盘高-盘低-收盘图；股票图；股价图

在经济界，一个比较流行的用法，是对时间序列进行聚合，计算每一个桶（bucket）里的四个值：first（open），last（close），maximum（high），minimal（low），即开盘-收盘-盘高-盘低，四个值。使用ohlc聚合函数可以得到这四个聚合结果：

In [111]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


### Upsampling and Interpolation

把一个低频度转换为高频度，是不需要进行聚合的。下面是一个周数据的DataFrame：

In [112]:
frame = pd.DataFrame(np.random.randn(2, 4), 
                     index=pd.date_range('1/1/2000', periods=2, freq='W-WED'), 
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,0.339717,-0.771501,0.086551,-0.923769
2000-01-12,-0.679238,0.122302,-0.507166,0.027078


当我们对这个数据进行聚合的的时候，每个组只有一个值，以及gap（间隔）之间的缺失值。在不使用任何聚合函数的情况下，我们使用asfreq方法将其转换为高频度：

In [113]:
df_daily = frame.resample('D').asfreq()
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,0.339717,-0.771501,0.086551,-0.923769
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-0.679238,0.122302,-0.507166,0.027078


假设我们想要用每周的值来填写非周三的部分。这种方法叫做填充（filling）或插值（interpolation），可以使用fillna或reindex方法来实现重采样：

In [114]:
frame.resample('D').ffill()

,Colorado,Texas,New York,Ohio
2000-01-05,0.339717,-0.771501,0.086551,-0.923769
2000-01-06,0.339717,-0.771501,0.086551,-0.923769
2000-01-07,0.339717,-0.771501,0.086551,-0.923769
2000-01-08,0.339717,-0.771501,0.086551,-0.923769
2000-01-09,0.339717,-0.771501,0.086551,-0.923769
2000-01-10,0.339717,-0.771501,0.086551,-0.923769
2000-01-11,0.339717,-0.771501,0.086551,-0.923769
2000-01-12,-0.679238,0.122302,-0.507166,0.027078


我们可以选择只对一部分的周期进行填写：

In [115]:
frame.resample('D').ffill(limit=2)

,Colorado,Texas,New York,Ohio
2000-01-05,0.339717,-0.771501,0.086551,-0.923769
2000-01-06,0.339717,-0.771501,0.086551,-0.923769
2000-01-07,0.339717,-0.771501,0.086551,-0.923769
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-0.679238,0.122302,-0.507166,0.027078


注意，新的日期索引不能与旧的有重叠：

In [116]:
frame.resample('W-THU').ffill()

,Colorado,Texas,New York,Ohio
2000-01-06,0.339717,-0.771501,0.086551,-0.923769
2000-01-13,-0.679238,0.122302,-0.507166,0.027078


#### Resampling with Periods

对周期的索引进行重采样的过程，与之前时间戳的方法相似：

In [117]:
frame = pd.DataFrame(np.random.randn(24, 4), 
                     index=pd.period_range('1-2000', '12-2001', freq='M'), 
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame[:5]

,Colorado,Texas,New York,Ohio
2000-01,-0.468856,0.584637,0.620118,1.154332
2000-02,-0.135512,-0.905260,-0.364922,0.061951
2000-03,0.516182,-0.444023,1.840488,1.229329
2000-04,0.736003,2.198893,-2.015712,1.932462
2000-05,0.374144,-0.856800,-1.291562,-0.223135


In [118]:
annual_frame = frame.resample('A-DEC').mean()
annual_frame

,Colorado,Texas,New York,Ohio
2000,-0.175252,-0.210286,-0.377797,0.241697
2001,0.187743,-0.403525,-0.231752,0.237262


增采样需要考虑的要多一些，比如在重采样前，选择哪一个时间跨度作为结束，就像asfreq方法那样。convertion参数默认是'start'，但也能用'end'：

In [119]:
# Q-DEC: Quarterly, year ending in December
annual_frame.resample('Q-DEC').ffill()

,Colorado,Texas,New York,Ohio
2000Q1,-0.175252,-0.210286,-0.377797,0.241697
2000Q2,-0.175252,-0.210286,-0.377797,0.241697
2000Q3,-0.175252,-0.210286,-0.377797,0.241697
2000Q4,-0.175252,-0.210286,-0.377797,0.241697
2001Q1,0.187743,-0.403525,-0.231752,0.237262
2001Q2,0.187743,-0.403525,-0.231752,0.237262
2001Q3,0.187743,-0.403525,-0.231752,0.237262
2001Q4,0.187743,-0.403525,-0.231752,0.237262


In [120]:
annual_frame.resample('Q-DEC', convention='end').ffill()

,Colorado,Texas,New York,Ohio
2000Q4,-0.175252,-0.210286,-0.377797,0.241697
2001Q1,-0.175252,-0.210286,-0.377797,0.241697
2001Q2,-0.175252,-0.210286,-0.377797,0.241697
2001Q3,-0.175252,-0.210286,-0.377797,0.241697
2001Q4,0.187743,-0.403525,-0.231752,0.237262


增采样比降采样的规则更严格一些：

* 降采样中，目标频度必须是原频度的子周期（subperiod）

* 增采样中，目标频度必须是原频度的母周期（superperiod）

如果不满足上面的规则，会报错。主要会影响到季度，年度，周度频度；例如，用Q-MAR定义的时间跨度只与A-MAR, A-JUN, A-SEP, A-DEC进行对齐（line up with）：

In [121]:
annual_frame.resample('Q-MAR').ffill()

,Colorado,Texas,New York,Ohio
2000Q4,-0.175252,-0.210286,-0.377797,0.241697
2001Q1,-0.175252,-0.210286,-0.377797,0.241697
2001Q2,-0.175252,-0.210286,-0.377797,0.241697
2001Q3,-0.175252,-0.210286,-0.377797,0.241697
2001Q4,0.187743,-0.403525,-0.231752,0.237262
2002Q1,0.187743,-0.403525,-0.231752,0.237262
2002Q2,0.187743,-0.403525,-0.231752,0.237262
2002Q3,0.187743,-0.403525,-0.231752,0.237262
